1、DataSet 是抽象类，不能实例化对象，主要是用于构造我们的数据集。

2、DataLoader 需要获取DataSet提供的索引[i]和len;用来帮助我们加载数据，比如说做shuffle(提高数据集的随机性)，batch_size,能拿出Mini-Batch进行训练。它帮我们自动完成这些工作。DataLoader可实例化对象。DataLoader is a class to help us loading data in Pytorch。

3、__getitem__目的是为支持下标(索引)操作。

1、需要mini_batch 就需要import DataSet和DataLoader。

2、继承DataSet的类需要重写init，getitem,len魔法函数。分别是为了加载数据集，获取数据索引，获取数据总量。

3、DataLoader对数据集先打乱(shuffle)，然后划分成mini_batch。

4、len函数的返回值 除以 batch_size 的结果就是每一轮epoch中需要迭代的次数。

5、inputs, labels = data中的inputs的shape是[32,8],labels 的shape是[32,1]。也就是说mini_batch在这个地方体现的。

6、diabetes.csv数据集给了下载地址，该数据集需和源代码放在同一个文件夹内。

In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
 
# prepare dataset

class DiabetesDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]        # shape(多少行，多少列)
        self.x_data = torch.from_numpy(xy[:, :-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
 
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
 
    def __len__(self):
        return self.len
 
 
dataset = DiabetesDataset('diabetes.csv')
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=0)    #num_workers 多线程
 
# design model using class
 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(8, 6)
        self.linear2 = torch.nn.Linear(6, 4)
        self.linear3 = torch.nn.Linear(4, 1)
        self.sigmoid = torch.nn.Sigmoid()
 
    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        return x
 
 
model = Model()
 
# construct loss and optimizer
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
 
# training cycle forward, backward, update
if __name__ == '__main__':
    for epoch in range(100):
        for i, data in enumerate(train_loader, 0):     # train_loader 是先shuffle后mini_batch
            inputs, labels = data
            y_pred = model(inputs)
            loss = criterion(y_pred, labels)
            print(epoch, i, loss.item())
 
            optimizer.zero_grad()
            loss.backward()
 
            optimizer.step()

0 0 0.8129358291625977
0 1 0.8556415438652039
0 2 0.778791606426239
0 3 0.8215760588645935
0 4 0.819486677646637
0 5 0.8173922300338745
0 6 0.7316904067993164
0 7 0.8144918084144592
0 8 0.8122555017471313
0 9 0.7964990735054016
0 10 0.8080813884735107
0 11 0.7535507082939148
0 12 0.7916620373725891
0 13 0.7902418971061707
0 14 0.737777590751648
0 15 0.7999390363693237
0 16 0.78597092628479
0 17 0.7235931158065796
0 18 0.7474836111068726
0 19 0.7113910913467407
0 20 0.7462306618690491
0 21 0.7574619054794312
0 22 0.7677301168441772
0 23 0.7492896914482117
1 0 0.7660875916481018
1 1 0.8092144727706909
1 2 0.7409369945526123
1 3 0.7726002931594849
1 4 0.7395392656326294
1 5 0.7914257049560547
1 6 0.7684878706932068
1 7 0.7567659616470337
1 8 0.7457980513572693
1 9 0.7845175266265869
1 10 0.7048247456550598
1 11 0.7530893087387085
1 12 0.7518683671951294
1 13 0.7697018980979919
1 14 0.7679742574691772
1 15 0.7123676538467407
1 16 0.7210223078727722
1 17 0.7914485335350037
1 18 0.7885479331

1、将原始数据集分为训练集和测试集。

2、对训练集进行批量梯度下降。

3、评估测试集的准确率。

In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split


# 读取原始数据，并划分训练集和测试集
raw_data = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32)
X = raw_data[:, :-1]
y = raw_data[:, [-1]]
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3)
Xtest = torch.from_numpy(Xtest)
Ytest = torch.from_numpy(Ytest)

# 将训练数据集进行批量处理
# prepare dataset

class DiabetesDataset(Dataset):
    def __init__(self, data,label):

        self.len = data.shape[0]     # shape(多少行，多少列)
        self.x_data = torch.from_numpy(data)
        self.y_data = torch.from_numpy(label)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


train_dataset = DiabetesDataset(Xtrain,Ytrain)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=0) #num_workers 多线程

# design model using class


class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(8, 6)
        self.linear2 = torch.nn.Linear(6, 4)
        self.linear3 = torch.nn.Linear(4, 2)
        self.linear4 = torch.nn.Linear(2, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        x = self.sigmoid(self.linear4(x))
        return x


model = Model()

# construct loss and optimizer
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


# training cycle forward, backward, update

def train(epoch):
    train_loss = 0.0
    count = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        y_pred = model(inputs)

        loss = criterion(y_pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        count = i

    if epoch%2000 == 1999:
        print("train loss:", train_loss/count,end=',')


def test():
    with torch.no_grad():
        y_pred = model(Xtest)
        y_pred_label = torch.where(y_pred>=0.5,torch.tensor([1.0]),torch.tensor([0.0]))
        acc = torch.eq(y_pred_label, Ytest).sum().item() / Ytest.size(0)
        print("test acc:", acc)

if __name__ == '__main__':
    for epoch in range(50000):
        train(epoch)
        if epoch % 2000 == 1999:
            test()

train loss: 0.6831228621304035,test acc: 0.6403508771929824
train loss: 0.6848700027912855,test acc: 0.6403508771929824
train loss: 0.6804093010723591,test acc: 0.6403508771929824
train loss: 0.6803558431565762,test acc: 0.6403508771929824
train loss: 0.6820343360304832,test acc: 0.6403508771929824
train loss: 0.6808787807822227,test acc: 0.6403508771929824
train loss: 0.6785527803003788,test acc: 0.6403508771929824
train loss: 0.5946997720748186,test acc: 0.7017543859649122
train loss: 0.4930460248142481,test acc: 0.7894736842105263
train loss: 0.4892833884805441,test acc: 0.7850877192982456
train loss: 0.48475037701427937,test acc: 0.793859649122807
train loss: 0.46029084734618664,test acc: 0.8026315789473685
train loss: 0.4602952115237713,test acc: 0.7894736842105263
train loss: 0.44837704487144947,test acc: 0.7719298245614035
train loss: 0.4521815776824951,test acc: 0.7719298245614035
train loss: 0.4421262601390481,test acc: 0.7850877192982456
train loss: 0.4392797900363803,test ac